# Travaux pratiques 1 

## Année: 2024-2025

## Objectifs
1. Renforcer les concepts à l'aide d'outils comme NumPy et scikit-learn.
2. Appliquer la logique propositionnelle et la tester sur des ensembles de données d'images.
3. Explorer la logique prédicative et la valider sur des ensembles de données d'images.
4. Comprendre les techniques d'analyse de texte, y compris le stemming, la lemmatisation et l'analyse morphologique.

### Exercice 1.0 [★]

Testez le notebook Python Jupyter [rappels](./TP0.ipynb) et familiarisez-vous avec les différentes méthodes des bibliothèques : numpy, scikit-learn, etc.

### Exercise 1.1 [★]

L'exercice consiste à tester la logique propositionnelle. Le fichier CSV (`image_data.csv`) contient des attributs tels que la couleur, la forme, la taille, la texture et la classification après le processus d'analyse d'image. Définissez les propositions ci-dessous et testez les expressions logiques basées sur ces attributs pour évaluer les relations au sein des données.

##### Étape 1 : Charger et Inspecter le Fichier CSV
1. Lire le fichier CSV dans un DataFrame pandas.
2. Afficher les premières lignes du DataFrame pour comprendre la structure.

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../../data/image_data.csv')

# Inspect the data
print(df.head())

#### Étape 2 : Définir des Propositions Basées sur les Données
1. Créer des propositions booléennes basées sur les colonnes du fichier CSV.
   - `P` : Vérifie si la couleur est bleue.
   - `Q` : Vérifie si la forme est un cercle.
   - `R` : Vérifie si la classification est un animal.
   - `S` : Vérifie si la taille est grande (définir un seuil pour 'grand').
   - `T` : Vérifie si la texture est rugueuse.
   - `U` : Vérifie si la classification est un véhicule.
   - `V` : Vérifie si la classification est un bâtiment.

#### Étape 3 : Définir les Expressions Logiques
Créer des expressions logiques basées sur les propositions définies à l'étape 2.
   - `expr1` : Si la couleur est bleue et la forme est un cercle, alors la classification est un animal.
   - `expr2` : Si la taille est grande et la texture est rugueuse, alors la classification est un véhicule.
   - `expr3` : Si la classification est un bâtiment, que la forme est un cercle, que la couleur est bleue et que la taille est grande, alors la classification est un bâtiment.

#### Étape 4 : Tester les Expressions
1. Tester les expressions logiques sur chaque ligne du DataFrame.
2. Afficher les résultats des expressions pour chaque ligne.

##### Étape 5 : Ajouter une Proposition Composée avec Négation et Disjonction

Créer une nouvelle expression complexe qui teste ce qui suit :
- Si l'objet n'est **pas bleu** ou **a une texture lisse**, alors il est classé comme **non un objet**.

##### Étape 6 : Compter les Lignes Satisfaisantes pour Chaque Expression

Compter le nombre de lignes où chaque expression logique est `True` et comparer les fréquences des propositions satisfaites.

### Exercice 1.2 [★]

##### Étape 1 : Installer et Importer Z3
- Installer la bibliothèque de solveur Z3 (`z3-solver`).
- Importer Z3 et se familiariser avec ses fonctions de base.

In [ ]:
!pip install z3-solver

In [ ]:
from z3 import *

##### Étape 2 : Définir les Attributs comme Variables de Logique du Premier Ordre
- Définir des variables pour chaque colonne (par exemple, `color`, `shape`, `size`).
- Spécifier les valeurs possibles (par exemple, `color` peut être bleu, rouge, vert, etc.).
- Définir des contraintes pour des attributs tels que **size** étant un entier et d'autres attributs étant des chaînes de caractères.

In [ ]:
# Declare Z3 variables
Color = String('color')
Shape = String('shape')
Size = Int('size')
Texture = String('texture')
Classification = String('classification')

#### Étape 3 : Encoder les Propositions Logiques en Logique du Premier Ordre
- Encoder les propositions fournies en utilisant la logique de Z3 (voir l'exercice 1.1).
- Exemple : Si l'objet est bleu et circulaire, alors il est classé comme un animal (`Implies(And(Color == "blue", Shape == "circle"), Classification == "animal")`).

In [ ]:
# Define constraints
valid_colors = Or(Color == "blue", Color == "red", Color == "green", Color == "yellow", Color == "purple")
valid_shapes = Or(Shape == "circle", Shape == "square", Shape == "triangle", Shape == "rectangle", Shape == "ellipse")
valid_size = Size >= 100  # Size constraint
valid_textures = Or(Texture == "polka dot", Texture == "smooth", Texture == "patterned", Texture == "rough")
valid_classifications = Or(Classification == "animal", Classification == "plant", Classification == "object", Classification == "vehicle", Classification == "building")

# Add these constraints to the solver
solver = Solver()
solver.add(valid_colors, valid_shapes, valid_size, valid_textures, valid_classifications)

##### Étape 4 : Résoudre pour la Satisfaisabilité

- Utiliser le solveur Z3 pour vérifier si les propositions sont satisfaisables.
- Afficher les résultats.

In [ ]:
# Check if the solver finds a solution that satisfies the constraints
if solver.check() == sat:
    print("The propositions are satisfiable.")
    model = solver.model()
    print(model)
else:
    print("The propositions are not satisfiable.")

##### Étape 5 : Ajouter des Contraintes Supplémentaires
- Ajouter une contrainte qui restreint certaines combinaisons, comme « si l'objet est vert, il ne peut pas être circulaire ».
- Ajouter une autre contrainte où « les objets à pois ne peuvent pas être des véhicules ».

## Exercice 1.3 [★★]

Lire le fichier CSV `image_data.csv` et définir les attributs comme des variables Z3 en fonction des données du fichier. Encoder les propositions logiques et les contraintes en utilisant la logique du premier ordre pour chaque ligne, puis résoudre pour la satisfaisabilité.

In [ ]:
import pandas as pd
from z3 import *

# Load CSV data
df = pd.read_csv('../../data/image_data.csv')

# Initialize Z3 solver
solver = Solver()

##### Étape 2 : Définir les Variables Z3 Dynamiquement à Partir des Données CSV
Pour chaque ligne du CSV, définir les attributs comme des variables Z3 et s'assurer que les types sont cohérents.

In [ ]:
# Define Z3 variables for each attribute dynamically for each row
for index, row in df.iterrows():
    color = String(f'color_{index}')
    shape = String(f'shape_{index}')
    size = Int(f'size_{index}')
    texture = String(f'texture_{index}')
    classification = String(f'classification_{index}')
    
    # Add constraints for valid values
    solver.add(Or(color == row['color'], shape == row['shape'], size == row['size'], texture == row['texture'], classification == row['classification']))


#### Étape 3 : Encoder des Propositions en Logique du Premier Ordre
- Écrire des propositions logiques pour chaque ligne, comme « si un objet est bleu et circulaire, alors il est classé comme un animal ».
- Utiliser l'exercice 1.1 et ajouter des propositions supplémentaires.

In [ ]:
# Example FOL for each row
for index, row in df.iterrows():
    expr1 = Implies(And(String(f'color_{index}') == "blue", String(f'shape_{index}') == "circle"), String(f'classification_{index}') == "animal")
    solver.add(expr1)


#### Étape 4 : Résoudre pour la Satisfaisabilité
Vérifier si les propositions logiques pour les données CSV sont satisfaisables.

#### Étape 5 : Ajouter des Contraintes Supplémentaires
- Inclure de nouvelles contraintes (par exemple, « les objets verts ne peuvent pas être circulaires ») et vérifier à nouveau le résultat.

In [ ]:
no_green_circle = Implies(String(f'color_{index}') == "green", String(f'shape_{index}') != "circle")
solver.add(no_green_circle)

#### Étape 6 : Introduire une Contrainte Insatisfaisable
- Ajouter une contrainte conflictuelle qui force un objet à avoir deux attributs mutuellement exclusifs (par exemple, être à la fois bleu et rouge).
- Vérifier la satisfaisabilité et montrer que le modèle devient insatisfaisable.

In [ ]:
# Add a conflicting constraint: Object must be both blue and red (which is impossible)
for index in range(len(df)):
    conflicting_constraint = And(String(f'color_{index}') == "blue", String(f'color_{index}') == "red")
    solver.add(conflicting_constraint)

# Check satisfiability after adding the conflicting constraint
if solver.check() == sat:
    print("The propositions are still satisfiable.")
else:
    print("The model is now unsatisfiable due to conflicting constraints.")


### Exercice 1.4 [★★]

Téléchargez cette page Web de Wikipedia : [https://fr.wikipedia.org/wiki/Paris](https://fr.wikipedia.org/wiki/Paris) et enregistrez le fichier au format HTML. Analysez la page Wikipedia en extrayant et en comptant les mots, les liens, les images, les nombres, les dates, les noms propres et les données structurées à partir des tableaux, tout en différenciant les sections et les paragraphes. Cela implique de télécharger le HTML, de le parser et d'identifier systématiquement le contenu pertinent. Écrivez un programme pour mettre en œuvre ces tâches :

1. **Télécharger le HTML** : Récupérer et enregistrer la page Wikipedia au format HTML.
2. **Charger le Contenu** : Lire et parser le fichier HTML pour analyse.
3. **Analyse des Mots** : Compter les occurrences de mots dans le texte.
4. **Extraction des Liens** : Identifier et catégoriser les liens internes et externes.
5. **Extraction d'Images** : Localiser les images et rassembler leurs URL et tailles.
6. **Extraction de Nombres et de Dates** : Identifier les nombres, les dates et les coordonnées géographiques.
7. **Noms Propres** : Extraire les noms de personnes et de lieux.
8. **Données des Tableaux** : Localiser et extraire les données des tableaux.
9. **Différenciation des Sections** : Identifier les sections et les paragraphes dans le contenu.

#### Analyse de la Page Wikipedia : Paris

Dans ce notebook, des tâches seront effectuées pour extraire et analyser divers éléments de la page Wikipedia de Paris.

##### Étape 1 : Télécharger la Page HTML
Tout d'abord, téléchargez le contenu HTML de la page Wikipedia spécifiée et enregistrez-le en tant que fichier HTML. Nous utilisons la bibliothèque `requests` pour gérer la requête HTTP. N'oubliez pas de vérifier le statut de la réponse pour confirmer que la page a été téléchargée avec succès.

In [ ]:
import requests

# URL of the Wikipedia page
url = "https://fr.wikipedia.org/wiki/Paris"

# Send a GET request to the URL
response = requests.get(url)

# Save the content as an HTML file
with open("paris.html", "w", encoding='utf-8') as file:
    file.write(response.text)

print("HTML page downloaded and saved as paris.html")


##### Étape 2 : Charger le Contenu HTML
Chargez le fichier HTML téléchargé pour une analyse plus approfondie.
- **Commentaire** : Parser le HTML est crucial pour extraire des données. Assurez-vous d'utiliser une bibliothèque comme BeautifulSoup qui peut naviguer efficacement dans la structure HTML.

Familiarisez-vous avec les méthodes de `BeautifulSoup` pour trouver des éléments dans le HTML, telles que `find()` et `find_all()`.

In [ ]:
from bs4 import BeautifulSoup

# Load the HTML file
with open("paris.html", "r", encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
print("HTML content loaded.")


##### Étape 3 : Extraire et Analyser les Mots
Comptez les occurrences de chaque mot sur la page.
- **Commentaire** : Envisagez de normaliser le texte en le convertissant en minuscules pour éviter de compter le même mot en différentes majuscules séparément. Nous utilisons des expressions régulières pour filtrer efficacement les caractères non alphanumériques lors de la séparation du texte en mots.

In [ ]:
from collections import Counter
import re

# Extract text from the HTML content
text = soup.get_text()

# Clean and split text into words
words = re.findall(r'\w+', text.lower())
word_count = Counter(words)

# Display the 10 most common words
print(word_count.most_common(10))


##### Étape 4 : Extraire les Liens
Identifiez tous les liens internes et externes de la page.

- **Commentaire** : Comprendre la différence entre les liens internes et externes est important pour la catégorisation.
- **Indice** : Vérifiez l'attribut `href` des balises d'ancrage (`<a>`) pour déterminer le type de lien.

##### Étape 5 : Extraire les Images et Leurs Tailles
Identifiez toutes les images sur la page et obtenez leurs tailles.

- **Commentaire** : Soyez conscient que les images ne sont pas toujours stockées dans le même format. Assurez-vous de construire les bonnes URLs pour elles.
- **Indice** : Vous devrez peut-être vérifier les attributs des balises `<img>` pour obtenir des informations supplémentaires, telles que la taille des images si disponible.

##### Étape 6 : Extraire les Nombres, Dates et Coordonnées Géographiques
Identifiez les nombres, dates et coordonnées géographiques dans le texte.

- **Commentaire** : Différents formats pour les dates et les nombres peuvent compliquer l'extraction. Considérez les diverses manières dont ces éléments peuvent apparaître sur la page.
- **Indice** : Utilisez des expressions régulières adaptées à des motifs spécifiques (par exemple, formats de date ou coordonnées géographiques) pour les identifier avec précision.

##### Étape 7 : Identifier les Noms Propres
Extraire les noms propres du texte.

- **Commentaire** : Les noms propres peuvent inclure des noms de personnes, de lieux et d'organisations. Les identifier correctement peut améliorer votre analyse des données.
- **Indice** : Utilisez des techniques de traitement du langage naturel (NLP), telles que la reconnaissance d'entités nommées, pour automatiser l'identification des noms propres.

##### Étape 8 : Extraire des Données Structurées (Tableaux)
Identifiez et extrayez des données des tableaux présents dans le HTML.

- **Commentaire** : Les tableaux contiennent souvent des données organisées qui peuvent être utiles pour l'analyse. Assurez-vous de capturer à la fois les cellules d'en-tête et les cellules de données.
- **Indice** : Familiarisez-vous avec la structure des tableaux HTML, y compris comment naviguer dans les lignes (`<tr>`) et les cellules (`<td>` et `<th>`).

##### Étape 9 : Différencier les Sections et les Paragraphes
Identifiez et séparez les sections et les paragraphes dans le contenu.

- **Commentaire** : Les sections aident à comprendre l'organisation du contenu. Reconnaître les différents niveaux de titres peut faciliter la navigation dans le contenu.
- **Indice** : Utilisez les balises appropriées (`<h1>`, `<h2>`, etc.) pour différencier les sections et assurez-vous de capturer leur contenu associé, comme les paragraphes.

## Exercice 1.5 [★★★]
Analysez le texte de la page Wikipedia téléchargée en appliquant le stemming, l'extraction d'n-grammes, l'étiquetage des parties du discours (PoS), la lemmatisation, l'analyse morphologique et la reconnaissance d'entités nommées. Comparez les résultats de NLTK et de spaCy pour évaluer leur efficacité dans les tâches d'analyse de texte.

### Prérequis

Assurez-vous d'avoir les bibliothèques nécessaires installées. Vous pouvez les installer en utilisant pip si ce n'est pas déjà fait :

In [ ]:
!pip install nltk spacy gensim wordcloud seaborn

In [ ]:
! python -m spacy download fr_core_news_sm  # For French language processing

#### Étape 1 : Charger la Page Wikipedia
Commencez par charger le fichier HTML que vous avez enregistré précédemment et extraire le texte.

In [ ]:
from bs4 import BeautifulSoup

# Load the HTML file
with open("paris.html", "r", encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, "html.parser")
text = soup.get_text()

#### Étape 2 : Appliquer des Algorithmes de Stemming
Utilisez les stemmers Porter et Snowball de NLTK pour réduire les mots du texte à leur racine.

In [ ]:
import nltk
from nltk.stem import PorterStemmer, SnowballStemmer
from collections import Counter
import re

# Tokenize and clean the text
words = re.findall(r'\w+', text.lower())

# Initialize stemmers
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer("english")

# Apply stemming
porter_stems = [porter_stemmer.stem(word) for word in words]
snowball_stems = [snowball_stemmer.stem(word) for word in words]

# Count unique stems
porter_stem_count = Counter(porter_stems)
snowball_stem_count = Counter(snowball_stems)

# Display the most common stems and count of unique stems
print("Most common Porter stems:", porter_stem_count.most_common(10))
print("Unique Porter stems count:", len(porter_stem_count))

print("Most common Snowball stems:", snowball_stem_count.most_common(10))
print("Unique Snowball stems count:", len(snowball_stem_count))

#### Étape 3 : Extraire des N-grammes
Générez et affichez les n-grammes les plus courants (1-grammes à 5-grammes) du texte.

#### Étape 4 : Étiquetage des Parties du Discours (PoS)
Utilisez NLTK ou spaCy pour effectuer l'étiquetage des parties du discours sur le texte.

#### Étape 5 : Lemmatisation
Appliquez la lemmatisation en utilisant NLTK ou spaCy.

#### Étape 6 : Analyse Morphologique
Utilisez spaCy pour effectuer une analyse morphologique sur le texte.

#### Étape 7 : Reconnaissance d'Entités Nommées (NER)
Utilisez spaCy pour identifier les entités nommées dans le texte.

#### Étape 8 : Distribution de Fréquence des Mots
Visualisez la distribution de fréquence des mots à l'aide de Matplotlib.

#### Étape 9 : Créer un Nuage de Mots

Générez un nuage de mots pour visualiser les mots les plus fréquents.

#### Étape 10 : Visualisation des Entités Nommées

Visualisez les entités nommées reconnues dans le texte à l'aide de Matplotlib.

#### Étape 11 : Visualisation des Noms les Plus Courants

Visualisez les noms les plus courants dans le texte, ce qui peut fournir des informations sur les principaux sujets abordés.